In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 

## 1.Get all links for zimuku items

In [2]:
## send the request
url = "http://www.zimuku.net/search?q=&t=onlyst"
result = requests.get(url)
print(result.status_code)

200


In [3]:
## get page number
c = result.content
soup = BeautifulSoup(c,"lxml")
pages =  soup.find('div',{'class':'pagination r clearfix'}).find('div')
page_num = pages.findAll('a')[-1].getText()
page_num = int(page_num)
print(page_num)


12993


In [4]:
## get download link from title link 
def get_down_link(url):
    #url = "http://www.zimuku.net/shooter/57265.html"
    try:
        result = requests.get(url)
        if result.status_code != 200:
            return None
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        down_link =  soup.find('li',{'class':'li dlsub'}).find('a').get('href')
        return down_link
    except:
        return None

In [5]:
## create doc_list to hold all links 
doc_list = {'title':[],'link':[]}
page_num = 100
## loop through each page
for p in range(1,page_num+1):
    try:
        page_url = url + '&p=' + str(p)
        result = requests.get(page_url)
        #print(result.status_code)
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        docs = soup.find('div',{'class':'box clearfix'}).findAll('div',{'class':'persub clearfix'})

        for doc in docs:

            ## get title 
            link_tag = doc.find('h1',{'class':'title'}).find('a')
            title = link_tag.getText().strip(' \t\n\r@').replace('\n','').replace('\r',' ')
            if title == '_':
                continue
            elif "/" in title:
                 title = title.split("/")[0]

            link='http://www.zimuku.net'+link_tag.get('href')

            ## get download link 
            down_link = get_down_link(link)

            doc_list['title'].append(title)
            doc_list['link'].append(down_link)
        except:
            continue

print(len(doc_list['link']))

18


In [6]:
## export links file 
df = pd.DataFrame(doc_list)
df['status']= 0
df.to_csv('zimuku_links.csv')
df.ix[0:5]

,link,title,status
0,http://shooter.zimuku.net/download/NTcwNjV8Z2x...,gladiator_2000_extended_subs_r3,0
1,http://shooter.zimuku.net/download/NTM3OTV8QW5...,An_Inconvenient_Truth_2006_Subs_R3,0
2,http://shooter.zimuku.net/download/Mzk2OHxkYXJ...,darkangel,0
3,http://shooter.zimuku.net/download/MjY2MjI3fGN...,chappie_tlr1_h1080p,0
4,http://shooter.zimuku.net/download/MjY2MjI2fE9...,Once_Upon_a_Time_in_America_1984_Extended_Blur...,0
5,http://shooter.zimuku.net/download/MjY2MjI1fDF...,Two.and.a.Half.Men.S12E04.720p.HDTV.X264-DIMEN...,0


## Now start downloading

In [7]:
## download 
def download_zip(url,folder):
    try:
        result = requests.get(url)
        fname = result.headers['Content-Disposition'].split('"')[1]
        res= result.content
        with open( folder + fname, "wb" ) as f :
                f.write( res )
        return True
    except:
        return False


In [8]:
df = pd.read_csv('zimuku_links.csv')
folder = 'data/'

In [9]:
#df=df.ix[:5]
for index,row in df.iterrows():
    try:
        link = row['link']
        response = download_zip(link,folder)
        df.loc[index,'status']= response
        print(index,response)
    except:
        df.loc[index,'status']= False

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
